#### Comments:
##### Function will create files with unit_tripnumber which starts with 0 for each user and increments 



In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os
import argparse

In [17]:
def extract_data(to_process,output_path):   
    df=pd.read_parquet(to_process)
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df['timestamp_diff']=df.groupby('unit')['timestamp'].diff().dt.components['hours'].reset_index(drop=True)
    trip_number=0
    temp=0
    for i,x in df.iterrows():
        current_trip = pd.DataFrame()
        if x['timestamp_diff']>7:
            unit=x['unit']
            if i==0:
                temp=unit
            if unit!=temp:
                trip_number=0
                temp=unit
            x=pd.DataFrame(x).T
            current_trip=pd.concat([current_trip,x]).reset_index(drop=True)
            current_trip['timestamp'] = current_trip['timestamp'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))
            current_trip.drop('timestamp_diff',axis=1,inplace=True)
            path=output_path
            file=f'{unit}_{trip_number}.csv'
            current_trip.to_csv(os.path.join(output_path, file), index=False)
            trip_number += 1


In [18]:
extract_data('E:\VS code\data','E:\\VS code\\results')

### Testing for one CSV

In [ ]:
import os
import requests
from dotenv import load_dotenv

# Load API key from environment variables
load_dotenv()
api_key = os.getenv("TOLLGURU_API_KEY")

url = f'https://apis.tollguru.com/toll/v2/gps-tracks-csv-upload?mapProvider=osrm&vehicleType=5AxlesTruck'
file_path = 'E:\\VS code\\results\\1000_0.csv'
headers = {'x-api-key': api_key, 'Content-Type': 'text/csv'}

with open(file_path, 'rb') as file:
    response = requests.post(url, data=file, headers=headers)
    print(response.status_code)
    print(response.json())

403
{'status': 403, 'message': 'Request denied. No active subscription found'}


### Function to iter over csv

In [ ]:
TOLLGURU_API_KEY = os.getenv('TOLLGURU_API_KEY')
TOLLGURU_API_URL = os.getenv('TOLLGURU_API_URL', 'https://apis.tollguru.com/toll/v2/gps-tracks-csv-upload?mapProvider=osrm&vehicleType=5AxlesTruck')

def send_request(file_path, output_dir):
    with open(file_path, 'rb') as file:
        headers = {'x-api-key': TOLLGURU_API_KEY, 'Content-Type': 'text/csv'}
        response = requests.post(TOLLGURU_API_URL, data=file, headers=headers)
        filename = os.path.basename(file_path)
        json_output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.json")
        with open(json_output_path, 'w') as json_file:
            json_file.write(response.text)

In [71]:
send_request('E:\\VS code\\results\\1000_0.csv','E:\\VS code\\results')

Comment fr Proocess 2 <br>
Keep Getting this error , couldn't proceed further.<br>
403
{'status': 403, 'message': 'Request denied. No active subscription found'}<br>
I have created and account in tollguru and got the correct apikey not sure why I'm getting no active subscripion

In [ ]:
input_dir = "E:\\VS code\\results"  # Update this to your directory containing CSV files
output_dir = "E:\\VS code\\results"  # Update this to your desired output directory

csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

for csv_file in csv_files:
    file_path = os.path.join(input_dir, csv_file)
    send_request(file_path, output_dir)